In [1]:
import keras
import pandas as pd
import numpy as np
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.callbacks import ModelCheckpoint
from keras.utils import np_utils
from keras.layers import Dense, Dropout, LSTM, Embedding, Reshape, Input, InputLayer, Concatenate
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential, Model

Using TensorFlow backend.


In [2]:
n_samples = 10000
sample_inputs = []
sample_outputs = []
n_tokens = 61

for n in range(n_samples):
    sample_in = []
    sample_out = []
    for line_number in range(3):
        i = np.random.randint(10,50)
        line = list(range(i, i+3))
        if line_number == 0:
            sample_in.append([line[0]] + line)
        else:
            sample_in.append([0] + line)
            
        sample_out.append(line + [60])
        
        
        #sample.append(np.arange(i, i+3))
    #samples.append(sample)
    sample_inputs.append(sample_in)
    sample_outputs.append(sample_out)
    
X = np_utils.to_categorical(sample_inputs, num_classes=n_tokens)
Y = np_utils.to_categorical(sample_outputs, num_classes=n_tokens)

In [3]:
latent_dim = 256

# Keras Model

inputs = [Input(shape=(None, n_tokens)) for i in range(3)]
lstms = [LSTM(latent_dim, return_state=True, return_sequences=True) for i in range(3)]

lstm_out, lstm_h, lstm_c = [None, None, None], [None, None, None], [None, None, None]
for i in range(3):
    if i > 0:
        lstm_out[i], lstm_h[i], lstm_c[i] = lstms[i](inputs[i], initial_state=[lstm_h[i-1], lstm_c[i-1]])
    else:
        lstm_out[i], lstm_h[i], lstm_c[i] = lstms[i](inputs[i])
        
denses = [Dense(n_tokens, activation='softmax') for i in range(3)]
outputs = [denses[i](lstm_out[i]) for i in range(3)]

model = Model(inputs, outputs)
model.compile(optimizer='rmsprop', loss='categorical_crossentropy')
model.summary(line_length=200)

________________________________________________________________________________________________________________________________________________________________________________________________________
Layer (type)                                                      Output Shape                                Param #                 Connected to                                                      
input_1 (InputLayer)                                              (None, None, 61)                            0                                                                                         
________________________________________________________________________________________________________________________________________________________________________________________________________
lstm_1 (LSTM)                                                     [(None, None, 256), (None, 256), (None, 256 325632                  input_1[0][0]                                                 

In [4]:
model.fit([X[:,0,:], X[:,1,:], X[:,2,:]], [Y[:,0,:], Y[:,1,:], Y[:,2,:]], batch_size=128, epochs=10)

Epoch 1/10
10000/10000 [==============================] - 5s 481us/step - loss: 8.8329 - dense_1_loss: 3.0621 - dense_2_loss: 2.8980 - dense_3_loss: 2.8728
Epoch 2/10
10000/10000 [==============================] - 3s 299us/step - loss: 6.3226 - dense_1_loss: 2.0566 - dense_2_loss: 2.1937 - dense_3_loss: 2.0722
Epoch 3/10
10000/10000 [==============================] - 3s 296us/step - loss: 3.7003 - dense_1_loss: 1.0568 - dense_2_loss: 1.4001 - dense_3_loss: 1.2434
Epoch 4/10
10000/10000 [==============================] - 3s 301us/step - loss: 2.5899 - dense_1_loss: 0.5801 - dense_2_loss: 1.0149 - dense_3_loss: 0.9948
Epoch 5/10
10000/10000 [==============================] - 3s 294us/step - loss: 2.1705 - dense_1_loss: 0.2613 - dense_2_loss: 0.9641 - dense_3_loss: 0.9451
Epoch 6/10
10000/10000 [==============================] - 3s 298us/step - loss: 1.9611 - dense_1_loss: 0.0803 - dense_2_loss: 0.9456 - dense_3_loss: 0.9351
Epoch 7/10
10000/10000 [==============================] - 3s 301

In [5]:
m = 1
print(sample_inputs[0])
np.argmax(model.predict([X[:m,0,:], X[:m,1,:], X[:m,2,:]]), axis=3)

[[11, 11, 12, 13], [0, 26, 27, 28], [0, 47, 48, 49]]


array([[[11, 12, 13, 60]],

       [[29, 27, 28, 60]],

       [[27, 48, 49, 60]]], dtype=int64)

In [6]:
generator_inputs = [Input(shape=(None, n_tokens), batch_shape=(1,None,n_tokens)) for i in range(3)]
generator_lstms = [LSTM(latent_dim, return_state=True, return_sequences=True, stateful=True) for i in range(3)]

generator_lstm_out, generator_lstm_h, generator_lstm_c = [None, None, None], [None, None, None], [None, None, None]
for i in range(3):
    if i > 0:
        generator_lstm_out[i], generator_lstm_h[i], generator_lstm_c[i] = generator_lstms[i](
            generator_inputs[i], initial_state=[
                generator_lstm_h[i-1], 
                generator_lstm_c[i-1]
            ]
        )
    else:
        generator_lstm_out[i], generator_lstm_h[i], generator_lstm_c[i] = generator_lstms[i](generator_inputs[i])
        
generator_denses = [Dense(n_tokens, activation='softmax') for i in range(3)]
generator_outputs = [generator_denses[i](generator_lstm_out[i]) for i in range(3)]

generator_model = Model(generator_inputs, generator_outputs)

In [7]:
generator_model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_4 (InputLayer)            (1, None, 61)        0                                            
__________________________________________________________________________________________________
lstm_4 (LSTM)                   [(1, None, 256), (1, 325632      input_4[0][0]                    
__________________________________________________________________________________________________
input_5 (InputLayer)            (1, None, 61)        0                                            
__________________________________________________________________________________________________
lstm_5 (LSTM)                   [(1, None, 256), (1, 325632      input_5[0][0]                    
                                                                 lstm_4[0][1]                     
          

In [8]:
for nb,layer in enumerate(model.layers):
    generator_model.layers[nb].set_weights(layer.get_weights())

In [83]:
#generator_model_lines = [Model(generator_inputs[i], generator_outputs[i])]
generator_line0 = Model(generator_inputs[0],  [generator_outputs[0], generator_lstm_h[0], generator_lstm_c[0]])

In [84]:
generator_line0.reset_states()
line = [10]

def f():
    while line[-1] != 60:
        yield np_utils.to_categorical(line[-1], num_classes=n_tokens).reshape((1,1,-1))
        
while line[-1] != 60:
    character, h, c = generator_line0.predict_generator(f(), steps=1)
    line.append(np.argmax(character))
    
line

[10, 10, 11, 12, 60]

In [85]:
generator_model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_40 (InputLayer)           (1, None, 61)        0                                            
__________________________________________________________________________________________________
lstm_34 (LSTM)                  [(1, None, 256), (1, 325632      input_40[0][0]                   
__________________________________________________________________________________________________
input_41 (InputLayer)           (1, None, 61)        0                                            
__________________________________________________________________________________________________
lstm_35 (LSTM)                  [(1, None, 256), (1, 325632      input_41[0][0]                   
                                                                 lstm_34[0][1]                    
          

In [86]:
generator_line0.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_40 (InputLayer)        (1, None, 61)             0         
_________________________________________________________________
lstm_34 (LSTM)               [(1, None, 256), (1, 256) 325632    
_________________________________________________________________
dense_25 (Dense)             (1, None, 61)             15677     
Total params: 341,309
Trainable params: 341,309
Non-trainable params: 0
_________________________________________________________________


In [90]:
generator_inputs

[<tf.Tensor 'input_40:0' shape=(1, ?, 61) dtype=float32>,
 <tf.Tensor 'input_41:0' shape=(1, ?, 61) dtype=float32>,
 <tf.Tensor 'input_42:0' shape=(1, ?, 61) dtype=float32>]

In [22]:
generator_h_input = Input(shape=(latent_dim,))
generator_c_input = Input(shape=(latent_dim,))

generator_line1_lstm = LSTM(latent_dim, return_state=True, return_sequences=True, stateful=True)

o1, h1, c1 = generator_line1_lstm(generator_inputs[1], initial_state=[generator_h_input, generator_c_input])
out1 = generator_denses[1](o1)

generator_line1 = Model([generator_inputs[1], generator_h_input, generator_c_input], [out1, h1, c1])

generator_line1_lstm.set_weights(generator_lstms[1].get_weights())

In [23]:
generator_line1.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_5 (InputLayer)            (1, None, 61)        0                                            
__________________________________________________________________________________________________
input_13 (InputLayer)           (None, 256)          0                                            
__________________________________________________________________________________________________
input_14 (InputLayer)           (None, 256)          0                                            
__________________________________________________________________________________________________
lstm_9 (LSTM)                   [(1, None, 256), (1, 325632      input_5[0][0]                    
                                                                 input_13[0][0]                   
          

In [30]:
generator_line1.reset_states()
#generator_line1_lstm.reset_states()
line = [0]
#h = np.array([np.zeros(256)])
#c = np.array([np.zeros(256)])

def f():
    while line[-1] != 60:
        yield [np_utils.to_categorical(line[-1], num_classes=n_tokens).reshape((1,1,-1)), h, c]
        
while line[-1] != 60:
    character, h, c = generator_line1.predict_generator(f(), steps=1)
    line.append(np.argmax(character))
    
line

TypeError: 'NoneType' object is not subscriptable

In [14]:
h

array([ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0

In [68]:
h.shape

(1, 256)

In [70]:
np_utils.to_categorical(line[-1], num_classes=n_tokens).reshape((1,1,-1)).shape

(1, 1, 61)

In [66]:
generator_line1.predict([np_utils.to_categorical(line[-1], num_classes=n_tokens).reshape((1,1,-1)), h, c])

InvalidArgumentError: You must feed a value for placeholder tensor 'input_34' with dtype float and shape [?,256]
	 [[Node: input_34 = Placeholder[dtype=DT_FLOAT, shape=[?,256], _device="/job:localhost/replica:0/task:0/device:CPU:0"]()]]